In [1]:
!pip install transformers -U
!pip install sentencepiece

Requirement already up-to-date: transformers in /usr/local/lib/python3.7/dist-packages (4.4.2)


In [15]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import AdamW
from transformers import Trainer, TrainingArguments


In [3]:
df = pd.read_pickle("./cleaned_dataset_df.pkl")
df

,name,author,categories,body/story,likesNumber,commentsNumber,timestamp of scraping,url
0,No Hard Feelings,A.dot Ram,"[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","Dear Winston,\nBy the time you’re reading this...",251,131,2020-11-16T13:51:25.961554,https://blog.reedsy.com/creative-writing-promp...
1,3:00 A.M.,Alby Carter,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","It’s late, and my head still hurts from the fi...",140,65,2020-11-16T13:51:26.814869,https://blog.reedsy.com/creative-writing-prom...
2,The Population,Julia Boddie,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",They rushed him to the hospital. Timothy laid ...,83,31,2020-11-16T13:51:27.692685,https://blog.reedsy.com/creative-writing-prom...
3,That Night,Eliza Ane,"[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",The door slammed against the wall as she threw...,82,79,2020-11-16T13:51:28.563709,https://blog.reedsy.com/creative-writing-prom...
4,A savior?,B. W.,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, ...",This has something to do with the novel that i...,72,260,2020-11-16T13:51:29.870689,https://blog.reedsy.com/creative-writing-prom...
...,...,...,...,...,...,...,...,...
3560,A Different Kind Of Ophalim,Azalea Young,"[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",Of Important Note:\nWith the creation of Yalib...,0,0,2021-02-03T23:41:12.424204,https://blog.reedsy.com/creative-writing-prom...
3561,The Miracle Maker,Reija Sillanpaa,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","TW: sexual abuse, familial abuse\n\nMy super-s...",0,0,2021-02-03T23:41:13.191278,https://blog.reedsy.com/creative-writing-prom...
3562,She Listens,Da. Blackmore,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",Tears.\nShe steps into the schoolyard silently...,0,0,2021-02-03T23:41:13.949157,https://blog.reedsy.com/creative-writing-prom...
3563,"Someone given a second chance, is someone give...",Nokwazi Mokoena,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",I cannot recall what was happening in my head ...,0,0,2021-02-03T23:41:14.786111,https://blog.reedsy.com/creative-writing-prom...


In [4]:
num_of_categories = len(df["categories"][0])
num_of_categories

24

In [7]:
#tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          num_labels = num_of_categories, 
                                          do_lower_case = True
                                          )

#Tokenizer examples:

In [6]:
print(tokenizer.tokenize("An example: apply, applied, application, applicable, responsible, managed, breaking!!!"))
print(tokenizer.tokenize("Wondering if stemming and lemmatization are on."))

['an', 'example', ':', 'apply', ',', 'applied', ',', 'application', ',', 'applicable', ',', 'responsible', ',', 'managed', ',', 'breaking', '!', '!', '!']
['wondering', 'if', 'stemming', 'and', 'le', '##mma', '##ti', '##zation', 'are', 'on', '.']


In [8]:
# model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-uncased') 
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

#Set a training mode:

In [ ]:
model.train()

In [12]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)

In [13]:
text_batch = ["I love Pixar.", "I don't care for Pixar."]
encoding = tokenizer(text_batch, return_tensors='pt', padding=True, truncation=True)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

In [14]:
labels = torch.tensor([1,0]).unsqueeze(0)
outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
loss = outputs.loss
loss.backward()
optimizer.step()

In [23]:
features = df["body/story"].to_list()
labels = df["categories"].to_list()
labels[-5:]

[[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [24]:
from sklearn.model_selection import train_test_split

features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.33, random_state=42)

# Creating custom Datasets

In [30]:
from torch.utils.data import Dataset

class TextClassificationDataset(Dataset):

    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        features = torch.from_numpy(self.features[idx])
        labels = torch.from_numpy(self.labels[idx])

        return features, labels[idx]

In [31]:
train_dataset = TextClassificationDataset(features_train, labels_train)
test_dataset = TextClassificationDataset(features_test, labels_test)

In [32]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,        # training dataset
    eval_dataset=test_dataset            # evaluation dataset
)

In [33]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='samples')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }